In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json
import time
import sqlite3
load_dotenv()
from keylog_analysis_helper import *
import difflib

Notes
- Code reconstruction may be incorrect, double check again
- Measures of progress:
    - Whether code compiles,
    - Ask llm if it progressed?
    - '# characters written
        - Those that are code and not (ask llm)


- does this portion make sense
- does this portion make sense to the solution 


Define model

In [3]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="models/gemini-1.5-pro-002",
  # generation_config=generation_config,
)

# chat_session = model.start_chat(
#   history=[
#   ]
# )

Save dataframes with diff

In [ ]:
def form_prompt():

    """
    You will help me determine whether a student has made progress towards completing their coding assignment.
    I will provide you with the code from the previous snapshot and the current snapshot. I will also provide you with the diff between the two files.
    
    Here is the previous snapshot for the file named {}
    Here is the current snapshot for the file named {}
    Here is the diff between the two files:
    {}
    
    Here is the previous snapshot for the file named {}
    Here is the current snapshot for the file named {}
    Here is the diff between the two files:
    {}

    The instructions for the assignment are as follows {}

    The autograder code that is used to score the student is as follows {}

    If you believe the student has progressed towards passing more tests in the autograder than simply answer with "1".
    If you believe it is the same or the student has regressed then answer with "0".
    If you believe it has regressed then answer with "-1".

    Simply answer with one of these three options. Do not give me an explanation.


    """

Define Loop

In [ ]:
differ = difflib.Differ()

def make_llm_comparisons(a3=True):

    # Read df based on assignment type
    if a3:
        df = pd.read_csv("KeylogDatasets/a3.csv")
        df.drop(columns=["Unnamed: 0", "autograder_error","bustersAgents.py_length","inference.py_length"], inplace=True)
    else:
        df = pd.read_csv("KeylogDatasets/a4.csv")
        df.drop(columns=["Unnamed: 0","autograder_error","NeuralNet.py_length","NeuralNetUtil.py_length"], inplace=True)
   
    df["Prediction"] = None
    df['Diff'] = None
    p_list = p_list_a3_log_analysis if a3 else p_list_a4_log_analysis
    assignment = 'a3' if a3 else 'a4'
    snapshots_dir = snapshots_dir_a3 if a3 else snapshots_dir_a4

    for p in p_list:
        df_temp = df[df["participant"] == p]

        # Create chat session
        chat_session = model.start_chat(
            history=[
                row["text"]
            ]
        )

        for idx, row in df_temp.iterrows()[1:]:
            print(row["participant"], row['snapshot'])

            # read snapshot file path
            # Read files
            prev_snapshot_dir = os.path.join(snapshots_dir, f'{assignment}_{p}_c{row['snapshot']-1}')
            current_snapshot_dir = os.path.join(snapshots_dir, f'{assignment}_{p}_c{row['snapshot']}')
            
            if a3:
                prev_file_1 = read_file_as_string(os.path.join(prev_snapshot_dir, "bustersAgents.py"))
                current_file_1 = read_file_as_string(os.path.join(current_snapshot_dir, "bustersAgents.py"))
                prev_file_2 = read_file_as_string(os.path.join(prev_snapshot_dir, "inference.py"))
                current_file_2 = read_file_as_string(os.path.join(current_snapshot_dir, "inference.py"))
            else:
                prev_file_1 = read_file_as_string(os.path.join(prev_snapshot_dir, "NeuralNet.py"))
                current_file_1 = read_file_as_string(os.path.join(current_snapshot_dir, "NeuralNet.py"))
                prev_file_2 = read_file_as_string(os.path.join(prev_snapshot_dir, "NeuralNetUtil.py"))
                current_file_2 = read_file_as_string(os.path.join(current_snapshot_dir, "NeuralNetUtil.py"))

            # Run diff
            diff_1 = differ.compare(prev_file_1, current_file_1)
            diff_2 = differ.compare(prev_file_2, current_file_2)
            

            # Read autograder code contents
            if a3:
                autograder_code = None
                assignment_instructions = None
            else:
                autograder_code = None
                assignment_instructions = None

            # Ask the llm
            
            prompt = form_prompt()


            response = chat_session.send_message()
            print(response)
            df.at[idx, "llm_response"] = response
            df.to_csv("KeylogDatasets/a3_llm.csv", index=False) if a3 else df.to_csv("KeylogDatasets/a4.csv", index=False)
            time.sleep(1)